In [1]:
# include should only be used once, serves as copy paste code purpose
# using should be used whenever contents in this module is called
# in individual modules, using other modules should consider hierarcy, 
# which is also related to how files are included in MAIN.

include("../src/Config_files/ConfigDataType.jl")
include("../src/SpatialAlgebra.jl")
include("../src/ConstructSystem.jl")
include("../src/UpdateSystem.jl")
include("../src/HERK.jl")

using ConfigDataType
using SpatialAlgebra
using ConstructSystem
using UpdateSystem
using HERK

In [2]:
# body = config_body(4,4,[1. 2. 3.;4. 5. 6.],1.)
nbody = 4
ndim = 2
config_body = ConfigBody(nbody)

 nbody=4
 nverts=4
 verts=[0.0 0.0; 1.0 0.0; 1.0 0.25; 0.0 0.25]
 ρ=0.01


In [3]:
njoint = nbody
config_joints = Vector{ConfigJoint}(njoint)

# set the first active joint
qJ_init = 
active_motion = Motions("oscillatory", [π/4, 1., 0.])
active_dof = Dof(3, "active", 0., 0., active_motion)
config_joints[1] = ConfigJoint(njoint, 1, "revolute",
                               zeros(Float64,6), zeros(Float64,6),
                               0, [active_dof], zeros(Float64,6))

# set the rest passive joint
for i = 2:njoint
    config_joints[i] = ConfigJoint(njoint, "revolute")
    config_joints[i].body1 = i-1
end

In [4]:
bodys = Vector{SingleBody}(nbody) # body system
for i = 1:nbody
    bodys[i] = AddBody(i, config_body) # add body
end

In [5]:
joints = Vector{SingleJoint}(njoint) # joint system
for i = 1:njoint
    joints[i] = AddJoint(i, config_joints[i]) # add joint
end

In [6]:
# assemble system to a chain
gravity = [0., 0., 0., ] # [0., -9.8, 0., ]
system = System(ndim, nbody, njoint, gravity)
bodys, joints, system = AssembleSystem!(bodys, joints, system)
system

ndim = 2, njoint = 4, nbody = 4
ndof = 24, nudof = 4, ncdof = 20, np = 3, na = 1
udof = [3, 9, 15, 21]
udof_p = [9, 15, 21]
udof_a = [3]
nudof_HERK = 3, ncdof_HERK = 21
udof_HERK = [9, 15, 21]
gravity = [0.0, 0.0, 0.0]


In [7]:
# test function UpdatePosition!
bodys, joints, system = UpdatePosition!(bodys, joints, system)

# test function UpdateVelocity!
for i = 1:nbody
    @assert length(bodys[i].v) == 6
    @assert length(bodys[i].Xp_to_b) == 36
    @assert length(joints[i].vJ) == 6
end
v = ones(Float64,24)
bodys, joints, system, vJ = UpdateVelocity!(bodys, joints, system, v)

# do some output
bodys[1]
bodys[2]
joints[1]

joint_id = 1, joint_type = revolute, pid = 0
shape1 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], shape2 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
nudof = 1, ncdof = 5, np = 0, na = 1
udof = [3], cdof = [1, 2, 4, 5, 6]
udof_p = Int64[], udof_a = [3]
i_udof_p = Int64[], i_udof_a = [1]
udofmap = [1]
cdofmap_HERK = [1, 2, 3, 4, 5, 6]
nudof_HERK = 0, ncdof_HERK = 6
udof_HERK = Int64[], cdof_HERK = [1, 2, 3, 4, 5, 6]
S = [0; 0; 1; 0; 0; 0]
T = [1 0 0 0 0; 0 1 0 0 0; 0 0 0 0 0; 0 0 1 0 0; 0 0 0 1 0; 0 0 0 0 1]
T_HERK = [1 0 0 0 0 0; 0 1 0 0 0 0; 0 0 1 0 0 0; 0 0 0 1 0 0; 0 0 0 0 1 0; 0 0 0 0 0 1]
joint_dof = ConfigDataType.Dof[ConfigDataType.Dof(3, "active", 0.0, 0.0, ConfigDataType.Motions("oscillatory", [0.785398, 1.0, 0.0]))]
Xj = [1.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0]
Xp_to_j = [1.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0

In [8]:
HERK.HERKFuncf(bodys, joints, system)
HERK.HERKFuncG(bodys, system)

21×24 Array{Float64,2}:
  1.0   0.0    0.0    0.0   0.0   0.0  …   0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   1.0    0.0    0.0   0.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0    1.0    0.0   0.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0    0.0    1.0   0.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0    0.0    0.0   1.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0    0.0    0.0   0.0   1.0  …   0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -1.0   0.0    0.0    0.0   0.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  -1.0    0.0    0.0   0.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0    0.0   -1.0   0.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0   -0.25   0.0  -1.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.25   0.0    0.0   0.0  -1.0  …   0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0    0.0    0.0   0.0   0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0    0.0    0.0   0.0   0.0      0.0  0.0 